# Lab 2

In [1]:
#pip install IP2Location

In [2]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [3]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [4]:
!ls /users/trush/CSC496/PageRank/Lab2

data  IP-COUNTRY.BIN  Lab2.ipynb


In [5]:
auth_log = sc.textFile("/users/trush/CSC496/PageRank/Lab2/data")
auth_log.take(5)

['Nov 15 00:07:01 submitty CRON[31671]: pam_unix(cron:session): session opened for user submitty_daemon by (uid=0)',
 'Nov 15 00:07:01 submitty CRON[31671]: pam_unix(cron:session): session closed for user submitty_daemon',
 'Nov 15 00:07:21 submitty sshd[31682]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.92.0.248  user=root',
 'Nov 15 00:07:23 submitty sshd[31682]: Failed password for root from 218.92.0.248 port 11673 ssh2',
 'Nov 15 00:07:27 submitty sshd[31682]: Failed password for root from 218.92.0.248 port 11673 ssh2']

## 1. How many failed attempts to access the server as root are there? List all countries from which these attempts where carried out.

In [6]:
'''
# Total failed attempts to login
total_attempts = auth_log.count()
print(total_attempts)
total_successful_attempts = auth_log.filter(lambda x: "CRON" in x)
print(total_successful_attempts.count())
total_failed_attempts = auth_log.filter(lambda x: "sshd" in x)
print(total_failed_attempts.count())

other_attempts = auth_log.filter(lambda x: "sshd" not in x and "CRON" not in x)
other_attempts.take(50)
'''
# Number of failed attempts to login as root
failed_attempts = auth_log.filter(lambda x: "root" in x and "sshd" in x)
failed_attempts.take(10)
#print(failed_attempts.count())

['Nov 15 00:07:21 submitty sshd[31682]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.92.0.248  user=root',
 'Nov 15 00:07:23 submitty sshd[31682]: Failed password for root from 218.92.0.248 port 11673 ssh2',
 'Nov 15 00:07:27 submitty sshd[31682]: Failed password for root from 218.92.0.248 port 11673 ssh2',
 'Nov 15 00:11:45 submitty sshd[31831]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=112.85.42.98  user=root',
 'Nov 15 00:11:47 submitty sshd[31831]: Failed password for root from 112.85.42.98 port 11028 ssh2',
 'Nov 15 00:11:51 submitty sshd[31831]: Failed password for root from 112.85.42.98 port 11028 ssh2',
 'Nov 15 00:16:28 submitty sshd[31926]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.92.0.165  user=root',
 'Nov 15 00:16:30 submitty sshd[31926]: Failed password for root from 218.92.0.165 port 19498 ssh2',
 'Nov 15 00:16:33 submitty sshd[3

In [22]:
import IP2Location
import re


def get_country_name(x):
    database = IP2Location.IP2Location(os.path.join("data", "/users/trush/CSC496/PageRank/Lab2/IP-COUNTRY.BIN"))
    ip = re.split(pattern='(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', string=x)[1]
    #print(ip)
    rec = database.get_all(ip).country_long
    #print(type(rec))
    return str(rec)

get_country_name('Nov 15 00:07:21 submitty sshd[31682]: pam_unix(sshd:auth): authentication failure; logname= uid=0 euid=0 tty=ssh ruser= rhost=218.92.0.248  user=root')

'China'

In [24]:
failed_root_countries = failed_attempts.map(lambda line: get_country_name(line))
#failed_root_countries.take(5)
failed_root_countries.distinct().collect()

['Korea, Republic of',
 'Viet Nam',
 'India',
 'Canada',
 'Ukraine',
 'Belgium',
 'Germany',
 'Thailand',
 'New Zealand',
 'Japan',
 'Romania',
 'Syrian Arab Republic',
 'United Arab Emirates',
 'Iran, Islamic Republic of',
 'Poland',
 'Serbia',
 'Malaysia',
 'Guatemala',
 'Uruguay',
 'Bangladesh',
 'Uganda',
 'Denmark',
 'Finland',
 'Tunisia',
 'Mongolia',
 'Cambodia',
 'Saudi Arabia',
 'Zimbabwe',
 'United Kingdom',
 '-',
 'Netherlands',
 'Mexico',
 'Australia',
 'Venezuela, Bolivarian Republic of',
 'Norway',
 'Chile',
 'Senegal',
 'Bolivia, Plurinational State of',
 'Algeria',
 'Mozambique',
 'Macao',
 'Panama',
 'Cyprus',
 'Slovenia',
 'Taiwan, Province of China',
 'Argentina',
 'Peru',
 'Spain',
 'Bulgaria',
 'Morocco',
 'Portugal',
 'Uzbekistan',
 'Ecuador',
 'Ethiopia',
 'Azerbaijan',
 'Costa Rica',
 'Austria',
 'China',
 'Pakistan',
 'Hungary',
 'Greece',
 'Indonesia',
 'Colombia',
 'Kazakhstan',
 'Tanzania, United Republic of',
 'Croatia',
 'Sweden',
 'Czech Republic',
 'Keny

There are a total of 75,630 failed attempts to access the server as root. A filter on the log files returns all entries with attempts to login as root (sshd log receives unauthorized login attempts, and CRON log represents successful logins). The function get_country_name splits each line in failed attempts to isolate the IP address and uses the IP2Location database to identify the country location. Then, failed_root_countries is assigned to a map of each country relative to each failed attempt's IP address. Finally, the distinct country names are outputted to identify all countries from which failed attempts were carried out.

## 2. How many failed attempts to access the server as non-root users are there? What are the attempted usernames? List all countries from which these attempts where carried out. 

In [ ]:
# Number of failed attempts to login as non-root
failed_nonroot = auth_log.filter(lambda x: "root" not in x and "sshd" in x and "user unknown" not in x)
#failed_nonroot.take(5)
print(failed_nonroot.count())

In [ ]:
def get_username(x):
    line = re.split(pattern=r'user ', string=x)[1]
    line = re.split(pattern=r' from', string=line)[0]
    line = re.split(pattern=r'(\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})', string=line)[0]
    return line

In [ ]:
failed_usernames = failed_nonroot.filter(lambda x: 'user ' in x)
#failed_usernames.take(5)
failed_nonroot_usernames = failed_usernames.map(lambda line: get_username(line))
#failed_nonroot_usernames.take(50)
failed_nonroot_usernames.distinct().collect()

In [ ]:
failed_nonroot_countries = failed_nonroot.map(lambda line: get_country_name(line))
#failed_nonroot_countries.take(5)
failed_nonroot_countries.distinct().collect()

There are 76,541 failed attempts to access the server as non-root. A filter on the log files identifies all non-root entries, (explain sshd) with known usernames. All entries with a user name are filtered from failed_nonroot and assigned to failed_usernames. The values are then mapped according to the username in each entry using the get_username function. Each distinct value is outputted as list that identifies every username associated with failed login attempts.

## 3. Through out the durations of the log files, which date has the highest number of attack attempts? Anything interesting about that particular date?

In [ ]:
def get_date(x):
    line = re.split(pattern=r'(Jan?|Feb?|Mar?|Apr?|May?|Jun?|Jul?|Aug?|Sep?|Oct?|Nov?|Dec?\s+\d{1,2})', string=x)[1] + re.split(pattern=r'(Jan?|Feb?|Mar?|Apr?|May?|Jun?|Jul?|Aug?|Sep?|Oct?|Nov?|Dec?\s+\d{1,2})', string=x)[2]
    line = re.split(pattern=r'\d{2}\:\d{2}\:\d{2}', string=line)[0]
    return line

In [ ]:
dates = failed_attempts.map(lambda line: (get_date(line), 1))
dates.take(5)
countsByKey = dates.countByKey()
countsByKey

The log files are mapped by the date which is formatted as one A-Z, two a-z, one or two whitespace characters, and two 0-9. The .countByKey() method outputs the number of attack attempts per date. The date with the most number of attacks is December 6, but it is important to note the high volume of attempts from December 2 through December 8. These dates are important because (EXPLAIN).